# LLamaindex
LlamaIndex is a framework for building context-augmented generative AI applications with LLMs including agents and workflows.

https://docs.llamaindex.ai/en/stable/

In [2]:
#! pip install "llama-index==0.11.15"
# !pip install llama-index-embeddings-openai
# !pip install llama-index-llms-openai
# !pip install llama-index-readers-file
# !pip install docx2txt
# !pip install python-dotenv

In [1]:
from dotenv import dotenv_values
import os

In [2]:
# read env file
ROOT_DIR = os.getcwd()
config = dotenv_values(os.path.join(ROOT_DIR, "keys", ".env"))


In [3]:
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding

# initialize the LLM
llm = OpenAI(model="gpt-4o", api_key=config.get("OPENAI_API_KEY"))

# initialize the embedding
embed_model = OpenAIEmbedding(model="text-embedding-ada-002", api_key=config.get("OPENAI_API_KEY"))

In [5]:
from llama_index.core import Settings

# global settings
Settings.llm = llm
Settings.embed_model = embed_model

In [7]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader

# load documents
data = SimpleDirectoryReader(input_dir=os.path.join(ROOT_DIR, "data"),required_exts=[".pdf"]).load_data()

# indexing documents using vector store
index = VectorStoreIndex.from_documents(data)

In [21]:
data[1]

Document(id_='7fddff88-fdb5-4f12-8537-e3cb5df53534', embedding=None, metadata={'page_label': '2', 'file_name': 'Parser Source 2.pdf', 'file_path': 'D:\\repos\\llamaindex\\data\\Parser Source 2.pdf', 'file_type': 'application/pdf', 'file_size': 1930169, 'creation_date': '2024-10-26', 'last_modified_date': '2024-10-25'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text="Aardvark Constructions Limited  \nDate  Company Profile  Company Number  \n09/08/2024  123456  \n \nDate 09/08/2024  AARDVCONST/MCPARTLA -C Page 1 Main Details  \n Name:  Aardvark Constructions Limited  \n QuickRef:  AARDVCONST  \n Country:  United Kingdom  \n Company Number:  123456  \n Incorporated:  20/10/2020  \n Company Type:  Limited by Shares  \n Company Status:  Active  \n Dissolved: 

In [13]:

# converting vector store to query engine
query_engine = index.as_query_engine(similarity_top_k=3)

# generating query response
response = query_engine.query("Appointments Board Positions list with names and other details of Aardvark Constructions Limited")
print(response)

The appointments to board positions at Aardvark Constructions Limited include the following:

1. Nicole Adams was appointed as Director on 10/09/2020 and later as Alternate Director on 15/03/2023. She resigned from her positions as Director on 31/08/2023 and as Alternate Director, Chief Executive, Company Secretary, Deputy Chief Executive, Designated Member, Directeur Conformité, Director, and Joint Secretary on 14/11/2023.

2. Tim Haines was appointed as Director on 13/11/2022.

3. ABNB AltBauNeu Baugesellschaft mbH was appointed as Company Secretary on 19/12/2022.

4. Trustme was appointed as Managing Member on 30/01/2023.

5. Mohammed Malek was appointed as Chair on 31/01/2023 and resigned from his positions as Chair, Chief Executive Officer, and Director on 10/04/2023.

6. Gordon Tatun was appointed as Director on 12/10/2023.

7. Brian Jenkins was appointed as Director on 29/11/2023.


In [17]:
from llama_index.core.memory import ChatMemoryBuffer
from llama_index.core.chat_engine import CondensePlusContextChatEngine

# creating chat memory buffer
memory = ChatMemoryBuffer.from_defaults(token_limit=4500)

# creating chat engine
chat_engine = CondensePlusContextChatEngine.from_defaults(index.as_retriever(),memory=memory,llm=llm)
prompt = """
Provide Main Details of the company Aardvark Constructions Limited. Including following details:
Name:
Country:
Company Number:
Incorporated:
Company Type:
Company Status:
Primary Addresses Registered Office:
Accounting Dates:
Confirmation Statement:
"""
# generating chat response
response = chat_engine.chat(prompt)
print(str(response))

Certainly! Here are the main details for Aardvark Constructions Limited:

- **Name:** Aardvark Constructions Limited
- **Country:** United Kingdom
- **Company Number:** 123456
- **Incorporated:** 20/10/2020
- **Company Type:** Limited by Shares
- **Company Status:** Active
- **Primary Addresses Registered Office:** 6 Chancery Road, London, WC2A 5DP, United Kingdom
- **Accounting Dates:**
  - Last Period End: 16/11/2022
  - Current Period End: 16/11/2024
  - Last Extended: 16/11/2022
- **Confirmation Statement:**
  - Last Signed: 17/02/2023
  - Filed: 17/02/2023
  - Next Overdue: 03/03/2023


In [18]:
prompt = """
From Management Details extract:
Managed By:
Managed By Email:
"""
response = chat_engine.chat(prompt)
print(str(response))

From the Management Details of Aardvark Constructions Limited:

- **Managed By:** Caroline McPartland
- **Managed By Email:** cmcpartland@diligent.com


In [19]:
prompt = """
Past Names of the Company with their period 
"""
response = chat_engine.chat(prompt)
print(str(response))

I'm sorry, but the documents provided do not contain any information about past names of Aardvark Constructions Limited or the periods during which they were used.


In [20]:
prompt = """
Appointments Board Positions list with names and other details
"""
response = chat_engine.chat(prompt)
print(str(response))

Here is a list of appointments to board positions for Aardvark Constructions Limited, along with the relevant details:

1. **Nicole Adams (PRIMARYNM)**
   - Position: Director
   - Appointment Date: 10/09/2020
   - Resignation Date: 31/08/2023

2. **Tim Haines (HAINES -T)**
   - Position: Director
   - Appointment Date: 13/11/2022

3. **ABNB AltBauNeu Baugesellschaft mbH (ABNBGERGMB)**
   - Position: Company Secretary
   - Appointment Date: 19/12/2022

4. **Trustme (TRUSTME)**
   - Position: Managing Member
   - Appointment Date: 30/01/2023

5. **Mohammed Malek (MALEK -M)**
   - Position: Chair
   - Appointment Date: 31/01/2023
   - Resignation Date: 10/04/2023

6. **Nicole Adams (ADAMS -N)**
   - Position: Alternate Director
   - Appointment Date: 15/03/2023
   - Resignation Date: 14/11/2023

7. **Gordon Tatun (TATUN -G)**
   - Position: Director
   - Appointment Date: 12/10/2023

8. **Brian Jenkins (JENKINS -B)**
   - Position: Director
   - Appointment Date: 29/11/2023
